Source: 


  @inproceedings{OrbitalTrajectories,
    url = {https://www.researchgate.net/publication/348929030_Modern_Numerical_Programming_with_Julia_for_Astrodynamic_Trajectory_Design},
    year = {2021},
    publisher = {AAS/AIAA},
    author = {Padilha, Dan and Dei Tos, Diogene Alessandro and Baresi, Nicola and Kawaguchi, Junichiro},
    title = {Modern Numerical Programming with Julia for Astrodynamic Trajectory Design},
    booktitle = {31st AAS/AIAA Space Flight Mechanics Meeting}
  }
  

In [1]:
using ModelingToolkit
using DifferentialEquations
using Plots

In [2]:
r(μ, pos) = @. norm((pos - [-μ, 0., 0.], pos - [1 - μ, 0., 0.]))

r (generic function with 1 method)

In [3]:
Ω_3(μ, (x, y, z)) = 0.5*(x^2+y^2) + sum(((1-μ), μ) ./ r(μ,(x,y,z))) + 0.5*μ*(1-μ)

Ω_3 (generic function with 1 method)

In [4]:
ω(μ, e, f, (x, y, z)) = (Ω_3(μ,(x,y,z)) - 0.5*z^2*e*cos(f)) / (1 + e*cos(f))

ω (generic function with 1 method)

In [5]:
abstract type AstrodynamicalModel end      # Super-type of all astrodynamical models
struct _ER3BP <: AstrodynamicalModel end   # Concrete representation of ER3BP equations

function ModelingToolkit.ODESystem(::Type{_ER3BP})
    @parameters μ e f               # Mass fraction, eccentricity, true anomaly
    @variables x(f) y(f) z(f)       # Position variables as functions of true anomaly
    @derivatives D'~f D2''~f        # Derivatives with respect to true anomaly
    @derivatives Dx'~x Dy'~y Dz'~z  # Derivatives with respect to position
    _ω = ω(μ, e, f, (x, y, z))      # Call math functions using symbolic variables
    return ODESystem([
            D2(x) ~ +2D(y) + Dx(_ω),
            D2(y) ~ -2D(x) + Dy(_ω),    # Equations of motion
            D2(z) ~ +Dz(_ω)
        ], f, [x, y, z], [μ, e])        # Independent variable, state variable, parameters

end

Example Julia code for constructing any arbitrary astrodynamical model, allowing the Julia
JIT compiler to automatically generate specialised, performant versions of its propagation
function directly from high-order differential equations.

In [6]:
struct _ER3BP_ODEFunctions{S,F,F2} <: AstrodynamicalModel
    ode_system::S
    ode_f::F
    ode_stm_f::F2
end

In [7]:
function (T::Type{<:AstrodynamicalModel})(args...; kwargs...)
    # Expand and lower model to a 1st-order system of differential equations
    _ODE = ODESystem(T, args...; kwargs...)
    _eqs = expand_derivatives.(equations(_ODE))
    eqs, states = ode_order_lowering(_eqs, independent_variable(_ODE), states(_ODE))
    ODE = ODESystem(simplify.(eqs), independent_variable(_ODE), states, parameters(_ODE))
    return ODEFunction(ODE) # Generate integration functions automatically
end